In [1]:


import networkx as nx
import osmnx as ox
from tqdm import tqdm

# import matplotlib.pyplot as plt
import city_tests
import graph_generator


In [2]:
def get_graph(city_id: str = 'R2555133') -> nx.Graph:
    gdf = ox.geocode_to_gdf(city_id, by_osmid=True)
    polygon_boundary = gdf.unary_union
    graph = ox.graph_from_polygon(polygon_boundary,
                                  network_type='drive',
                                  simplify=True)
    G = nx.Graph(graph)
    H = nx.Graph()
    # Добавляем рёбра в новый граф, копируя только веса
    for u, d in G.nodes(data=True):
        H.add_node(u, x=d['x'], y=d['y'])
    for u, v, d in G.edges(data=True):
        H.add_edge(u, v, length=d['length'])
    del city_id, gdf, polygon_boundary, graph, G
    return H


In [ ]:
def test_graph(graph: nx.Graph, name: str, city_id: str, points: list[tuple[int, int]] = None,
               resolutions: list[float] = None, pos=2, logs=True, alg='dijkstra') -> CityResult:
    max_alpha = 1 if resolutions is None else max(resolutions)
    delta = max_alpha / 40
    if resolutions is None:
        resolutions = []
        resolutions += [i / 10 for i in range(1, 10, 1)]
        resolutions += [i for i in range(1, 10, 1)]
        resolutions += [i for i in range(10, 50, 2)]
        resolutions += [i for i in range(50, 100, 5)]
        resolutions += [i for i in range(100, 500, 10)]
        resolutions += [i for i in range(500, 1000, 50)]
        resolutions += [i for i in range(1000, 5000, 200)]
    if points is None:
        N: int = 1000
        points = [get_node_for_initial_graph_v2(graph) for _ in trange(N, desc='generate points')]
    else:
        N = len(points)

    has_coords = 'x' in [d for u, d in graph.nodes(data=True)][0]

    usual_results = get_usual_result(graph, points, alg=alg)

    data = {
        
    }

    alphas = set()
    for r in resolutions:
        start = time.time()
        community = graph_generator.resolve_communities(graph, r)
        # print(len(community) / len(graph.nodes))
        if len(community) < 5:
            continue
        a = len(community) / len(graph.nodes)
        has = False
        for curr in alphas:
            if abs(curr - a) < delta:
                has = True
                break
        if has or a > max_alpha:
            if logs:
                tqdm.write(f'alpha: {a} -- skip')
            if a == 1 and 1 in alphas or a > max_alpha:
                break
            else:
                continue
        alphas.add(a)
        layer, build_communities, build_additional, build_centroid_graph = generate_layer(graph, r,
                                                                                          has_coordinates=has_coords,
                                                                                          communities=community)
        tmp = test_layer(points, layer, alg=alg)
        total = time.time() - start
        text = """
                name:           {}
                alpha:          {:4f}
                total time:     {:.3f}
                prepare time:   {:.3f} 
                    build_communities:      {:.3f}
                    build_additional:       {:.3f}
                    build_centroid_graph:   {:.3f}
                pfa time:       {:.3f}
            """.format(name, a, total, total - tmp[0], build_communities, build_additional, build_centroid_graph,
                       tmp[0])
        if logs:
            tqdm.write(text)
        result.points_results.append(generate_result(usual_results, tmp, r, layer))


    result.save()
    if logs:
        s = [p.speed_up[0] for p in result.points_results]
        indx = np.argmax(s)
        max_s = s[indx]
        print(alg + ' usual time:', result.points_results[indx].absolute_time[0] * max_s)
        print(alg + ' hpfa time:', result.points_results[indx].absolute_time[0])
        print(alg + ' max_speedUp:', max(s))
        print(alg + ' mean_err:', np.mean(result.points_results[np.argmax(s)].errors),
              np.std(result.points_results[0].errors))
        print(alg + ' max_err:', np.max(result.points_results[np.argmax(s)].errors))
    return result


In [3]:
def calculate(data):
    cities = data[0]
    points_number = data[1]
    NUMBER = data[2]
    THREADS = data[3]

    for name, id in tqdm(cities, position=NUMBER):
        G = get_graph(id)

        points = [graph_generator.get_node_for_initial_graph_v2(G) for _ in
                  range(points_number)]

        Q = G.copy()
        for u in Q.nodes:
            if u in Q[u]:
                Q.remove_edge(u, u)
        test_graph(Q,
                              f'{name}',
                              id,
                              points=points, pos=NUMBER, logs=False)

In [4]:
total = 2
points_number = 500

cities = {
        # 'ASHA': 'R13470549',
        # 'KRG': 'R4676636',
        # 'EKB': 'R6564910',
        # 'BARCELONA': 'R347950',
        'PARIS': 'R71525',
        # 'Prague': 'R435514',
        # 'MSK': 'R2555133',
        # 'SBP': 'R337422',
        # 'SINGAPORE': 'R17140517',
        # 'BERLIN': 'R62422',
        # 'ROME': 'R41485',
        # 'LA': 'R207359',
        # 'DUBAI': 'R4479752',
        # 'RIO': 'R2697338',
        # 'DELHI': 'R1942586',
        # 'KAIR': 'R5466227'
}

total_len = len(cities)
l = list(cities.items())
data = [[l[i: total_len: total], points_number, i + 1, total] for i in range(total)]

calculate(data[0])

2



  0%|          | 0/1 [00:07<?, ?it/s]

KeyboardInterrupt

